# 02. 周波数フィルタリング

FFTを使った周波数領域でのフィルタリング技術を学びます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq
from scipy import signal
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. 基本的なフィルタの実装

In [ ]:
def create_frequency_filter(frequencies, filter_type, cutoff_freq, sample_rate, transition_width=50):
    """
    周波数領域フィルタを作成
    
    Parameters:
    frequencies: 周波数軸
    filter_type: 'lowpass', 'highpass', 'bandpass', 'bandstop'
    cutoff_freq: カットオフ周波数（bandpass/bandstopの場合は[low, high]のリスト）
    sample_rate: サンプリングレート
    transition_width: 遷移帯域の幅
    """
    filter_response = np.ones_like(frequencies)
    
    if filter_type == 'lowpass':
        # ローパスフィルタ
        mask = np.abs(frequencies) > cutoff_freq
        filter_response[mask] = 0
        
        # 遷移帯域の追加（滑らかな減衰）
        transition_mask = (np.abs(frequencies) > cutoff_freq - transition_width/2) & \
                         (np.abs(frequencies) <= cutoff_freq + transition_width/2)
        transition_freqs = np.abs(frequencies)[transition_mask]
        filter_response[transition_mask] = 0.5 * (1 + np.cos(np.pi * (transition_freqs - cutoff_freq + transition_width/2) / transition_width))
        
    elif filter_type == 'highpass':
        # ハイパスフィルタ
        mask = np.abs(frequencies) < cutoff_freq
        filter_response[mask] = 0
        
        # 遷移帯域
        transition_mask = (np.abs(frequencies) >= cutoff_freq - transition_width/2) & \
                         (np.abs(frequencies) < cutoff_freq + transition_width/2)
        transition_freqs = np.abs(frequencies)[transition_mask]
        filter_response[transition_mask] = 0.5 * (1 - np.cos(np.pi * (transition_freqs - cutoff_freq + transition_width/2) / transition_width))
        
    elif filter_type == 'bandpass':
        # バンドパスフィルタ
        low_freq, high_freq = cutoff_freq
        mask = (np.abs(frequencies) < low_freq) | (np.abs(frequencies) > high_freq)
        filter_response[mask] = 0
        
    elif filter_type == 'bandstop':
        # バンドストップフィルタ
        low_freq, high_freq = cutoff_freq
        mask = (np.abs(frequencies) >= low_freq) & (np.abs(frequencies) <= high_freq)
        filter_response[mask] = 0
    
    return filter_response

# テスト信号の生成
sample_rate = 1000
duration = 2.0
t = np.linspace(0, duration, int(sample_rate * duration), False)

# 複数の周波数成分を持つ信号 + ノイズ
signal_clean = (np.sin(2 * np.pi * 50 * t) +    # 50Hz
               0.7 * np.sin(2 * np.pi * 150 * t) + # 150Hz  
               0.5 * np.sin(2 * np.pi * 300 * t))  # 300Hz

# ノイズを追加
np.random.seed(42)
noise = 0.3 * np.random.normal(0, 1, len(t))
signal_noisy = signal_clean + noise

# FFT
fft_signal = fft(signal_noisy)
frequencies = fftfreq(len(signal_noisy), 1/sample_rate)

# 異なるフィルタを適用
filter_types = ['lowpass', 'highpass', 'bandpass', 'bandstop']
filter_params = [
    200,           # ローパス: 200Hz以下
    100,           # ハイパス: 100Hz以上
    [120, 180],    # バンドパス: 120-180Hz
    [140, 160]     # バンドストップ: 140-160Hz
]

plt.figure(figsize=(20, 15))

# 元の信号
plt.subplot(5, 3, 1)
plt.plot(t[:500], signal_clean[:500], 'b-', label='クリーン信号', alpha=0.7)
plt.plot(t[:500], signal_noisy[:500], 'r-', label='ノイズ付き信号')
plt.title('元の信号')
plt.ylabel('振幅')
plt.legend()
plt.grid(True)

# 元の信号のスペクトル
plt.subplot(5, 3, 2)
positive_mask = frequencies >= 0
plt.plot(frequencies[positive_mask], np.abs(fft_signal)[positive_mask], 'b-', linewidth=2)
plt.title('元の信号のスペクトル')
plt.ylabel('振幅')
plt.grid(True)
plt.xlim(0, 400)

for i, (filter_type, cutoff) in enumerate(zip(filter_types, filter_params)):
    # フィルタ作成
    filter_response = create_frequency_filter(frequencies, filter_type, cutoff, sample_rate)
    
    # フィルタ適用
    filtered_fft = fft_signal * filter_response
    filtered_signal = np.real(ifft(filtered_fft))
    
    # フィルタ特性の表示
    plt.subplot(5, 3, (i+1)*3 + 1)
    plt.plot(frequencies[positive_mask], filter_response[positive_mask], 'g-', linewidth=2)
    plt.title(f'{filter_type.capitalize()}フィルタ特性')
    plt.ylabel('ゲイン')
    plt.grid(True)
    plt.xlim(0, 400)
    plt.ylim(0, 1.1)
    
    # フィルタ後のスペクトル
    plt.subplot(5, 3, (i+1)*3 + 2)
    plt.plot(frequencies[positive_mask], np.abs(fft_signal)[positive_mask], 'b--', 
            alpha=0.5, label='元のスペクトル')
    plt.plot(frequencies[positive_mask], np.abs(filtered_fft)[positive_mask], 'r-', 
            linewidth=2, label='フィルタ後')
    plt.title('フィルタ後のスペクトル')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)
    plt.xlim(0, 400)
    
    # フィルタ後の時間信号
    plt.subplot(5, 3, (i+1)*3 + 3)
    plt.plot(t[:500], signal_noisy[:500], 'b--', alpha=0.5, label='元の信号')
    plt.plot(t[:500], filtered_signal[:500], 'r-', linewidth=2, label='フィルタ後')
    plt.title('フィルタ後の時間信号')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)
    
    if i == len(filter_types) - 1:
        plt.xlabel('時間 (秒)')
        plt.subplot(5, 3, (i+1)*3 + 1)
        plt.xlabel('周波数 (Hz)')
        plt.subplot(5, 3, (i+1)*3 + 2)
        plt.xlabel('周波数 (Hz)')

plt.tight_layout()
plt.show()

## 2. ノイズ除去フィルタ

In [ ]:
def adaptive_noise_filter(signal, noise_threshold_db=-20):
    """
    適応的ノイズ除去フィルタ
    スペクトルの強度に基づいてノイズを除去
    """
    # FFT
    fft_signal = fft(signal)
    amplitude = np.abs(fft_signal)
    
    # dB変換
    amplitude_db = 20 * np.log10(amplitude + 1e-10)
    
    # 最大値を基準としたしきい値
    max_amplitude_db = np.max(amplitude_db)
    threshold = max_amplitude_db + noise_threshold_db
    
    # ノイズマスクの作成
    noise_mask = amplitude_db < threshold
    
    # ソフトフィルタリング（急激なカットオフを避ける）
    filter_response = np.ones_like(amplitude)
    filter_response[noise_mask] = 0.1  # 完全に0にせず、10%残す
    
    # フィルタ適用
    filtered_fft = fft_signal * filter_response
    filtered_signal = np.real(ifft(filtered_fft))
    
    return filtered_signal, filter_response, amplitude_db, threshold

# 様々なノイズレベルでテスト
noise_levels = [0.1, 0.3, 0.5, 0.8]
threshold_values = [-15, -20, -25, -30]  # dB

# クリーンな音楽的信号の生成
music_signal = (np.sin(2 * np.pi * 220 * t) +           # A3
               0.5 * np.sin(2 * np.pi * 220 * 2 * t) +   # 2倍音
               0.3 * np.sin(2 * np.pi * 220 * 3 * t) +   # 3倍音
               0.7 * np.sin(2 * np.pi * 330 * t))        # E4

plt.figure(figsize=(18, 12))

for i, (noise_level, threshold_db) in enumerate(zip(noise_levels, threshold_values)):
    # ノイズを追加
    np.random.seed(i + 10)
    noise = noise_level * np.random.normal(0, 1, len(t))
    noisy_signal = music_signal + noise
    
    # ノイズ除去
    cleaned_signal, filter_resp, amplitude_db, threshold = adaptive_noise_filter(
        noisy_signal, threshold_db)
    
    # 結果の表示
    plt.subplot(4, 4, i*4 + 1)
    plt.plot(t[:300], music_signal[:300], 'g-', alpha=0.7, label='クリーン')
    plt.plot(t[:300], noisy_signal[:300], 'r-', alpha=0.8, label=f'ノイズレベル: {noise_level}')
    plt.title(f'ノイズレベル: {noise_level}')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(4, 4, i*4 + 2)
    frequencies = fftfreq(len(noisy_signal), 1/sample_rate)
    positive_mask = frequencies >= 0
    plt.plot(frequencies[positive_mask], amplitude_db[positive_mask], 'b-', alpha=0.7)
    plt.axhline(y=threshold, color='r', linestyle='--', label=f'しきい値: {threshold_db} dB')
    plt.title('スペクトルとしきい値')
    plt.ylabel('振幅 (dB)')
    plt.legend()
    plt.grid(True)
    plt.xlim(0, 500)
    
    plt.subplot(4, 4, i*4 + 3)
    plt.plot(frequencies[positive_mask], filter_resp[positive_mask], 'g-', linewidth=2)
    plt.title('フィルタ応答')
    plt.ylabel('ゲイン')
    plt.grid(True)
    plt.xlim(0, 500)
    plt.ylim(0, 1.1)
    
    plt.subplot(4, 4, i*4 + 4)
    plt.plot(t[:300], noisy_signal[:300], 'r--', alpha=0.5, label='ノイズあり')
    plt.plot(t[:300], cleaned_signal[:300], 'b-', linewidth=2, label='ノイズ除去後')
    plt.plot(t[:300], music_signal[:300], 'g:', alpha=0.8, label='元のクリーン信号')
    plt.title('ノイズ除去結果')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)
    
    if i == len(noise_levels) - 1:
        plt.xlabel('時間 (秒)')
        plt.subplot(4, 4, i*4 + 2)
        plt.xlabel('周波数 (Hz)')
        plt.subplot(4, 4, i*4 + 3)
        plt.xlabel('周波数 (Hz)')
        plt.subplot(4, 4, i*4 + 4)
        plt.xlabel('時間 (秒)')

plt.tight_layout()
plt.show()

## 3. 位相を考慮したフィルタリング

In [ ]:
def apply_phase_preserving_filter(signal, cutoff_freq, filter_type='lowpass'):
    """
    位相を保持するフィルタリング
    前向きと後ろ向きの両方向でフィルタリングして位相歪みを除去
    """
    # Butterworth フィルタの設計
    nyquist = sample_rate / 2
    normalized_cutoff = cutoff_freq / nyquist
    
    if filter_type == 'lowpass':
        b, a = signal.butter(4, normalized_cutoff, btype='low')
    elif filter_type == 'highpass':
        b, a = signal.butter(4, normalized_cutoff, btype='high')
    
    # 通常のフィルタリング（位相歪みあり）
    filtered_normal = signal.filtfilt(b, a, signal, method='gust')
    
    # 一方向フィルタリング（位相歪みあり）
    filtered_oneway = signal.lfilter(b, a, signal)
    
    return filtered_normal, filtered_oneway, b, a

# テスト信号: インパルス応答とステップ応答を観察
# インパルス信号
impulse = np.zeros(1000)
impulse[500] = 1.0

# ステップ信号
step = np.zeros(1000)
step[500:] = 1.0

# 方形波信号
t_short = np.linspace(0, 1, 1000, False)
square_wave = signal.square(2 * np.pi * 5 * t_short)  # 5Hz方形波

test_signals = [impulse, step, square_wave]
signal_names = ['インパルス', 'ステップ', '方形波']

cutoff_freq = 50  # Hz

plt.figure(figsize=(18, 15))

for i, (test_signal, name) in enumerate(zip(test_signals, signal_names)):
    # フィルタリング実行
    filtered_normal, filtered_oneway, b, a = apply_phase_preserving_filter(
        test_signal, cutoff_freq, 'lowpass')
    
    # 元の信号
    plt.subplot(3, 4, i*4 + 1)
    plt.plot(test_signal, 'b-', linewidth=2)
    plt.title(f'元の信号: {name}')
    plt.ylabel('振幅')
    plt.grid(True)
    
    # 一方向フィルタリング結果
    plt.subplot(3, 4, i*4 + 2)
    plt.plot(test_signal, 'b--', alpha=0.5, label='元の信号')
    plt.plot(filtered_oneway, 'r-', linewidth=2, label='一方向フィルタ')
    plt.title('一方向フィルタリング')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)
    
    # 双方向フィルタリング結果
    plt.subplot(3, 4, i*4 + 3)
    plt.plot(test_signal, 'b--', alpha=0.5, label='元の信号')
    plt.plot(filtered_normal, 'g-', linewidth=2, label='双方向フィルタ')
    plt.title('双方向フィルタリング')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)
    
    # 比較
    plt.subplot(3, 4, i*4 + 4)
    plt.plot(test_signal, 'b-', alpha=0.7, label='元の信号', linewidth=2)
    plt.plot(filtered_oneway, 'r--', alpha=0.8, label='一方向', linewidth=2)
    plt.plot(filtered_normal, 'g-', alpha=0.8, label='双方向', linewidth=2)
    plt.title('比較')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)
    
    if i == len(test_signals) - 1:
        for j in range(4):
            plt.subplot(3, 4, i*4 + j + 1)
            plt.xlabel('サンプル')

plt.tight_layout()
plt.show()

# フィルタの周波数応答を表示
w, h = signal.freqz(b, a, worN=8000, fs=sample_rate)

plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(w, 20 * np.log10(abs(h)), 'b-', linewidth=2)
plt.axvline(x=cutoff_freq, color='r', linestyle='--', label=f'カットオフ: {cutoff_freq} Hz')
plt.title('フィルタの振幅応答')
plt.xlabel('周波数 (Hz)')
plt.ylabel('振幅 (dB)')
plt.grid(True)
plt.legend()
plt.xlim(0, 200)

plt.subplot(2, 1, 2)
angles = np.unwrap(np.angle(h))
plt.plot(w, angles, 'g-', linewidth=2)
plt.axvline(x=cutoff_freq, color='r', linestyle='--', label=f'カットオフ: {cutoff_freq} Hz')
plt.title('フィルタの位相応答')
plt.xlabel('周波数 (Hz)')
plt.ylabel('位相 (ラジアン)')
plt.grid(True)
plt.legend()
plt.xlim(0, 200)

plt.tight_layout()
plt.show()

print("位相歪みの影響:")
print("- 一方向フィルタリング: 位相遅延が発生し、信号が遅れる")
print("- 双方向フィルタリング: 位相歪みが相殺され、遅延なし")
print("- 特にインパルスやステップ応答で違いが顕著に現れる")

## 4. カスタム周波数応答フィルタ

In [ ]:
def create_custom_filter(frequencies, frequency_response_points):
    """
    ユーザー定義の周波数応答を持つフィルタを作成
    
    Parameters:
    frequencies: 周波数軸
    frequency_response_points: [(freq1, gain1), (freq2, gain2), ...]
    """
    # 周波数応答の補間
    freq_points = [point[0] for point in frequency_response_points]
    gain_points = [point[1] for point in frequency_response_points]
    
    # 正の周波数の応答を補間
    positive_freqs = np.abs(frequencies)
    filter_response = np.interp(positive_freqs, freq_points, gain_points)
    
    return filter_response

# 様々なカスタムフィルタを作成
custom_filters = {
    'イコライザー': [(0, 1.0), (100, 1.5), (200, 0.5), (300, 2.0), (400, 0.8), (500, 1.0)],
    'ノッチフィルタ': [(0, 1.0), (145, 1.0), (150, 0.0), (155, 1.0), (500, 1.0)],
    'シェルビング': [(0, 0.3), (100, 0.3), (150, 1.0), (500, 1.0)],
    'マルチバンド': [(0, 0.5), (50, 1.0), (100, 0.2), (200, 1.5), (300, 0.3), (500, 1.0)]
}

# 楽器をシミュレートした複雑な信号
instrument_signal = np.zeros_like(t)
for harmonic in range(1, 8):  # 7倍音まで
    freq = 110 * harmonic  # A2の倍音
    amplitude = 1.0 / harmonic  # 高次倍音は減衰
    instrument_signal += amplitude * np.sin(2 * np.pi * freq * t)

# ノイズを少し追加
np.random.seed(123)
instrument_signal += 0.1 * np.random.normal(0, 1, len(t))

plt.figure(figsize=(20, 12))

# 元の信号のスペクトル
fft_original = fft(instrument_signal)
frequencies = fftfreq(len(instrument_signal), 1/sample_rate)
positive_mask = frequencies >= 0

plt.subplot(3, 5, 1)
plt.plot(t[:200], instrument_signal[:200], 'b-', linewidth=2)
plt.title('元の楽器信号')
plt.ylabel('振幅')
plt.grid(True)

plt.subplot(3, 5, 6)
plt.plot(frequencies[positive_mask], np.abs(fft_original)[positive_mask], 'b-', linewidth=2)
plt.title('元のスペクトル')
plt.ylabel('振幅')
plt.grid(True)
plt.xlim(0, 800)

for i, (filter_name, response_points) in enumerate(custom_filters.items()):
    # カスタムフィルタを作成
    filter_response = create_custom_filter(frequencies, response_points)
    
    # フィルタを適用
    filtered_fft = fft_original * filter_response
    filtered_signal = np.real(ifft(filtered_fft))
    
    # フィルタ特性を表示
    plt.subplot(3, 5, i + 2)
    freq_points = [point[0] for point in response_points]
    gain_points = [point[1] for point in response_points]
    plt.plot(freq_points, gain_points, 'ro-', markersize=6, label='設計点')
    plt.plot(frequencies[positive_mask], filter_response[positive_mask], 'g-', 
            linewidth=2, label='補間結果')
    plt.title(f'{filter_name}フィルタ')
    plt.ylabel('ゲイン')
    plt.legend()
    plt.grid(True)
    plt.xlim(0, 500)
    plt.ylim(0, 2.5)
    
    # フィルタ後のスペクトル
    plt.subplot(3, 5, i + 7)
    plt.plot(frequencies[positive_mask], np.abs(fft_original)[positive_mask], 'b--', 
            alpha=0.5, label='元')
    plt.plot(frequencies[positive_mask], np.abs(filtered_fft)[positive_mask], 'r-', 
            linewidth=2, label='フィルタ後')
    plt.title('フィルタ後のスペクトル')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)
    plt.xlim(0, 800)
    
    # フィルタ後の時間信号
    plt.subplot(3, 5, i + 12)
    plt.plot(t[:200], instrument_signal[:200], 'b--', alpha=0.5, label='元')
    plt.plot(t[:200], filtered_signal[:200], 'r-', linewidth=2, label='フィルタ後')
    plt.title('フィルタ後の信号')
    plt.xlabel('時間 (秒)')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)

# 元の信号の時間表示（最下段左）
plt.subplot(3, 5, 11)
plt.plot(t[:200], instrument_signal[:200], 'b-', linewidth=2)
plt.title('元の信号（時間）')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.grid(True)

plt.tight_layout()
plt.show()

## 練習問題

1. 音楽の特定の楽器だけを抽出するバンドパスフィルタを設計してみましょう
2. エコー効果を除去するためのフィルタを作成してみましょう
3. 電源周波数（50Hz/60Hz）のハムノイズを除去するノッチフィルタを実装してみましょう